In [111]:
import pandas as pd

**Import and prepare the data**

In [112]:
def prepare_data(coin):
    df = pd.read_csv(f'../../crypto_prices/coin_{coin}.csv', index_col=0).reset_index()
    df.Date = pd.to_datetime(df.Date)
    df['dayofweek'] = df.Date.dt.dayofweek
    df['dayofmonth'] = df.Date.dt.day
    df['year_month'] = df.Date.dt.strftime("%Y-%m")
    df['avg_daily_price']= (df.High + df.Low) / 2
    return df

In [113]:
def calculate_profit_dca(df, frequency, purchase_year_month = '2018-01',\
                        purchase_day_of_week = 0, purchase_day_of_month = [0]):
    purchase_usd = 10
    if frequency == 'monthly':
        freq_cond = (df.dayofmonth.isin(purchase_day_of_month))
    if frequency == 'weekly':
        freq_cond = (df.dayofweek == purchase_day_of_week)

    purchase = df[(df.year_month >= purchase_year_month) & freq_cond]
    purchased_amount = (purchase_usd/purchase.avg_daily_price).sum()
    investment_amount = purchase_usd * len(purchase)
    investment_value = df[-1:].avg_daily_price * purchased_amount
    return (investment_value / investment_amount)

In [114]:
dfs = {}
coins = ['Bitcoin', 'Ethereum']

**Monthly**

In [115]:
days= []
monthly_profit = {}
for i in range(1,32):
    days.append(i)

for coin in coins:
    df = prepare_data(coin)
    monthly_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(1,32):    
        monthly_profit[coin].append(calculate_profit_dca(df, 'monthly', purchase_day_of_month=[i]).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [116]:
monthly_profit['days']=days
monthly_df = pd.DataFrame.from_dict(monthly_profit).set_index('days')
dfs['monthly'] = monthly_df
monthly_df.head()

,Bitcoin,Ethereum
days,,
1,4.037992,8.587175
2,3.999434,8.473401
3,3.958840,8.426079
4,3.984029,8.464631
5,3.989482,8.458709


**Weekly**

In [117]:
days= []
weekly_profit = {}
for i in range(0,7):
    days.append(i)

for coin in coins:
    df = prepare_data(coin)
    weekly_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(0,7):    
        weekly_profit[coin].append(calculate_profit_dca(df, 'weekly', purchase_day_of_week=i).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [118]:
weekly_profit['days']=days
weekly_df = pd.DataFrame.from_dict(weekly_profit).set_index('days')
dfs['weekly'] = weekly_df
weekly_df.head()

,Bitcoin,Ethereum
days,,
0,4.063660,8.676294
1,4.060661,8.687917
2,4.067146,8.719436
3,4.075425,8.754547
4,4.092533,8.792059


**Bi-weekly**

In [119]:
days= []
biweekly_profit = {}
for i in range(1,32):
    for j in range(i+1,32):
        days.append(f"{i}-{j}")

for coin in coins:
    df = prepare_data(coin)
    biweekly_profit[coin] = []
    print(f"Calculating profit for coin {coin}...")
    for i in range(1,32):
        for j in range(i+1,32):    
            biweekly_profit[coin].append(calculate_profit_dca(df, 'weekly', purchase_day_of_month=[i,j]).iloc[0])

Calculating profit for coin Bitcoin...
Calculating profit for coin Ethereum...


In [120]:
biweekly_profit['days']=days
biweekly_df = pd.DataFrame.from_dict(biweekly_profit).set_index('days')
dfs['biweekly'] = biweekly_df
biweekly_df.head()

,Bitcoin,Ethereum
days,,
1-2,4.06366,8.676294
1-3,4.06366,8.676294
1-4,4.06366,8.676294
1-5,4.06366,8.676294
1-6,4.06366,8.676294


In [127]:
coin_describe_values = {}
index = 'weekly'
for coin in coins:
    describe_values = {}
    for index, df in dfs.items():
        describe_values[index] = df.describe()[coin].values
    coin_describe_values[coin] = describe_values

In [137]:
coin_describe_dfs = {}
for coin in coins:
    coin_describe_dfs[coin] = pd.DataFrame\
                              .from_dict(coin_describe_values[coin], orient='index', columns=df.describe().index)

In [146]:
coin_describe_dfs['Bitcoin'].transpose()

,monthly,weekly,biweekly
count,31.000000,7.000000,4.650000e+02
mean,4.072101,4.070960,4.063660e+00
std,0.055323,0.010633,8.891350e-16
min,3.958840,4.060661,4.063660e+00
25%,4.040130,4.065123,4.063660e+00
50%,4.072303,4.067146,4.063660e+00
75%,4.114573,4.073069,4.063660e+00
max,4.157543,4.092533,4.063660e+00


In [145]:
coin_describe_dfs['Ethereum'].transpose()

,monthly,weekly,biweekly
count,31.000000,7.000000,4.650000e+02
mean,8.725376,8.719264,8.676294e+00
std,0.175149,0.040730,1.778270e-15
min,8.426079,8.676294,8.676294e+00
25%,8.580040,8.694342,8.676294e+00
50%,8.738615,8.703827,8.676294e+00
75%,8.869161,8.736991,8.676294e+00
max,9.013523,8.792059,8.676294e+00


* The montly DAC has provided the highest profits, but presents also the lowest profits.
* The biweekly approach shows to provide the less volatile profit over time
* The weekly approach has provided slightly higher maximum profit compared to biweekly but with similar minimums

We conclude that the best approach for those aiming at maximising the profit should use a monthly approach. As long as the right day in the month is picked to invest, since the risk is to lower the profit if the wrong day is picked. A following analysis will try to identify the best day of month to invest for a specific coin.
For a more stable profit, biweekly seems to provide more constant profits, regardless of the days on which the investment is made.